# Baseline with mse loss with wide cnn modified + adam
#### Convert histone mark signals and use deep CNN for training.

#### NOTE: Need to activate genomelake environment before this code. Simply type 'genomelake' in terminal.

In [1]:
%env CUDA_VISIBLE_DEVICES=6,7
import os, sys
sys.path.append("..")
import random
# custom file path package
from data import Data_Directories
# custom utility package
from utils.compute_util import *
# package for genomic data
from pybedtools import Interval, BedTool
from genomelake.extractors import ArrayExtractor, BigwigExtractor
# package for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats.stats import pearsonr,spearmanr
import tensorflow as tf

env: CUDA_VISIBLE_DEVICES=6,7


Using TensorFlow backend.


In [2]:
window_size = 5001
process_all = False
sample_num = 1000

In [3]:
# retrieve data
data = Data_Directories()
print data.intervals.keys()
print data.input_atac['day0'].keys()
print data.output_histone['day0'].keys()

['day6', 'day3', 'day0']
['100', '140']
['H3K27me3', 'H3K4me1', 'H3K27ac']


In [4]:
# get intervals for day0 data
day0_intervals = list(BedTool(data.intervals['day0']))
print '# of Intervals Extracted for day0: {}'.format(len(day0_intervals))

# of Intervals Extracted for day0: 267226


In [5]:
# create an ArrayExtractor for ATAC-seq for day0 with 140 base pairs
bw_140bp_day0 = ArrayExtractor(data.input_atac['day0']['140'])
print 'Finished extracting bigwig for day0, 140bp'

Finished extracting bigwig for day0, 140bp


In [6]:
# create a BigWigExtractor for histone makr 'H3K27ac' for day0
bw_histone_mark_day0 = BigwigExtractor(data.output_histone['day0']['H3K27ac'])
print 'Finished extracting bigwig for day0, 140bp'

Finished extracting bigwig for day0, 140bp


In [7]:
# normalize day0 intervals
normalized_day0_intervals = [normalize_interval(interval, window_size) for interval in day0_intervals if normalize_interval(interval, window_size)]
print 'Finished normalizing day0 intervals!'

Finished normalizing day0 intervals!


In [8]:
assert (len(day0_intervals)==len(normalized_day0_intervals))
print "Examples of original intervals"
print [(int(_interval.start)+int(_interval[-1]), [int(_interval.start), int(_interval.end)])
       for _interval in day0_intervals[:3]]
print "Examples of normalized intervals with window size of {}".format(window_size)
print [([int(_interval.start), int(_interval.end)])
       for _interval in  normalized_day0_intervals[:3]]

Examples of original intervals
[(123412027, [123411855, 123412989]), (123411941, [123411855, 123412989]), (131908564, [131908487, 131910071])]
Examples of normalized intervals with window size of 5001
[[123409527, 123414528], [123409441, 123414442], [131906064, 131911065]]


In [9]:
atac_seq_day0 = bw_140bp_day0(normalized_day0_intervals)
print atac_seq_day0.shape

(267226, 5001, 5)


In [10]:
#TODO: put this into utils if possible
def prune_invalid_intervals(intervals, bigwig_file):
    for _interval in intervals[:]:
        try:
            bigwig_file([_interval])
        except:
            intervals.remove(_interval)
            pass
        
print "Before pruning day0: {}".format(len(normalized_day0_intervals))
prune_invalid_intervals(normalized_day0_intervals, bw_140bp_day0)
print "After pruning day0: {}".format(len(normalized_day0_intervals))

Before pruning day0: 267226
After pruning day0: 267226


In [11]:
print "Dimension of ATAC-seq signal: {}".format(bw_140bp_day0(normalized_day0_intervals[:1]).shape)

Dimension of ATAC-seq signal: (1, 5001, 5)


In [12]:
print "Dimension of histone mark signal: {}".format(bw_histone_mark_day0(normalized_day0_intervals[:1]).shape)

Dimension of histone mark signal: (1, 5001)


In [13]:
# replace nan values with zeros and convert it to p-values
histone_mark_day0 = np.nan_to_num(bw_histone_mark_day0(normalized_day0_intervals))
print histone_mark_day0.shape

(267226, 5001)


In [14]:
histone_mark_day0 = np.expand_dims(histone_mark_day0, axis=2)
print histone_mark_day0.shape

(267226, 5001, 1)


In [15]:
print "Example histone mark signal"
print "\tRaw value: {}".format(bw_histone_mark_day0(normalized_day0_intervals[:1])[0][:5].reshape(-1))

Example histone mark signal
	Raw value: [ 0.97687  0.97687  0.97687  0.97687  0.97687]


In [16]:
from keras.layers import Input, Dense, Conv1D, Dropout, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from keras import optimizers
from keras import metrics
from keras import losses
from keras import backend as K
from keras.callbacks import Callback, TensorBoard, ReduceLROnPlateau, ModelCheckpoint

In [17]:
dropout_rate = 0.5
# parameters for first conv layer
hidden_filters_1 = 32
hidden_kernel_size_1 = window_size
# parameters for second conv layer
output_filters = 1
output_kernel_size = 32
# parameters for training
batch_size = 128
num_epochs = 300
evaluation_freq = 10

In [18]:
'''
Baseline CNN (based on KERAS functional API)
'''
inputs = Input(shape=(window_size, 5, ))
x = Conv1D(
    filters=hidden_filters_1,
    kernel_size=hidden_kernel_size_1,
    padding='same',
    activation='relu',
    strides=1)(inputs)
x = Dropout(dropout_rate)(x)
# x = BatchNormalization(momentum=0.8)(x)

outputs = Conv1D(
    filters=output_filters,
    kernel_size=output_kernel_size,
    padding='same',
    activation='linear',
    strides=1
    )(x)

model = Model(inputs=inputs, outputs=outputs)

In [19]:
# Print out model summary
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5001, 5)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 5001, 32)          800192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 5001, 32)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 5001, 1)           1025      
Total params: 801,217
Trainable params: 801,217
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
def pearson(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(np.multiply(xm,ym))
    r_den = K.sqrt(np.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den
    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return K.square(r)

In [21]:
# Helper functions for writing the scores into bigwig file
from itertools import izip
from itertools import groupby
import subprocess

def interval_key(interval):
    return (interval.chrom, interval.start, interval.stop)

def merged_scores(scores, intervals, merge_type):
    # A generator that returns merged intervals/scores
    # Scores should have shape: #examples x #categories x #interval_size
    # Second dimension can be omitted for a 1D signal
    signal_dims = scores.ndim - 1
    assert signal_dims in {1, 2}

    # Only support max for now
    assert merge_type == 'max'
    score_first_dim = 1 if signal_dims == 1 else scores.shape[1]

    dtype = scores.dtype

    sort_idx, sorted_intervals = \
        zip(*sorted(enumerate(intervals),
                    key=lambda item: interval_key(item[1])))
    sorted_intervals = BedTool(sorted_intervals)

    # Require at least 1bp overlap
    # Explicitly convert to list otherwise it will keep opening a file when
    # retrieving an index resulting in an error (too many open files)
    interval_clust = list(sorted_intervals.cluster(d=-1))
    for _, group in groupby(izip(sort_idx, interval_clust),
                            key=lambda item: item[1].fields[-1]):
        idx_interval_pairs = list(group)
        group_idx, group_intervals = zip(*idx_interval_pairs)

        if len(idx_interval_pairs) == 1:
            yield group_intervals[0], scores[group_idx[0], ...]
        else:
            group_chrom = group_intervals[0].chrom
            group_start = min(interval.start for interval in group_intervals)
            group_stop = max(interval.stop for interval in group_intervals)

            # This part needs to change to support more merge_types (e.g. mean)
            group_score = np.full((score_first_dim, group_stop - group_start),
                                  -np.inf, dtype)
            for idx, interval in idx_interval_pairs:
                slice_start = interval.start - group_start
                slice_stop = slice_start + (interval.stop - interval.start)
                group_score[..., slice_start:slice_stop] = \
                    np.maximum(group_score[..., slice_start:slice_stop],
                               scores[idx, ...])
            if signal_dims == 1:
                group_score = group_score.squeeze(axis=0)
            yield Interval(group_chrom, group_start, group_stop), group_score
            
def interval_score_pairs(intervals, scores, merge_type):
    return (izip(intervals, scores) if merge_type is None
            else merged_scores(scores, intervals, merge_type))

def _write_1D_deeplift_track(scores, intervals, file_prefix, merge_type='max',
                             CHROM_SIZES='/mnt/data/annotations/by_release/hg19.GRCh37/hg19.chrom.sizes'):
    assert scores.ndim == 2

    bedgraph = file_prefix + '.bedGraph'
    bigwig = file_prefix + '.bw'

    print 'Writing 1D track of shape: {}'.format(scores.shape)
    print 'Writing to file: {}'.format(bigwig)

    with open(bedgraph, 'w') as fp:
        for interval, score in interval_score_pairs(intervals, scores,
                                                    merge_type):
            chrom = interval.chrom
            start = interval.start
            for score_idx, val in enumerate(score):
                fp.write('%s\t%d\t%d\t%g\n' % (chrom,
                                               start + score_idx,
                                               start + score_idx + 1,
                                               val))
    print 'Wrote bedgraph.'

    try:
        output = subprocess.check_output(
            ['wigToBigWig', bedgraph, CHROM_SIZES, bigwig],
            stderr=subprocess.STDOUT)
        print 'wigToBigWig output: {}'.format(output)
    except subprocess.CalledProcessError as e:
        print 'wigToBigWig terminated with exit code {}'.format(
            e.returncode)
        print 'output was:\n' + e.output

    print 'Wrote bigwig.'

In [22]:
model_dir = os.path.join("models", "mse_lowlr_window4000")
log_dir = os.path.join("logs", "mse_lowlr_window4000")
srv_dir = os.path.join("/users", "jesikmin", "mse_lowlr_window4000")
#srv_dir = os.path.join("/srv", "www", "kundaje", "jesikmin", "mse_lowlr_window4000")
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
if not os.path.exists(srv_dir):
    os.makedirs(srv_dir)

In [23]:
# setting an adam optimizer with 1.0 clip norm
adam = optimizers.Adam(lr=1e-3, clipnorm=1.)

print "Compiling a model with adam optimizer"
model.compile(loss=losses.mean_squared_error,
              optimizer=adam,
              metrics=[pearson, metrics.mse, metrics.mae])

Compiling a model with adam optimizer


In [24]:
# CallBack: tensorboard with grad histogram
tensorboard = TensorBoard(log_dir=log_dir,
                          histogram_freq=1,
                          batch_size=batch_size,
                          write_grads=True)

In [25]:
# CallBack: reduce learning rate when validation loss meets plateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.96,
                              patience=5,
                              min_lr=1e-7)

In [26]:
# CallBack: evaluate model for every evaulation epoch
class EvaluateModel(Callback):
    def __init__(self, X_test, y_test):
        self.X_test, self.y_test = X_test, y_test
        self.epochs = 0

    def on_epoch_end(self, batch, logs={}):
        self.epochs += 1
        # evaluate the model
        if self.epochs % evaluation_freq == 0:
            scores = model.evaluate(X_test, y_test)
            print zip(model.metrics_names, scores)
            test_prediction = model.predict(X_test)
            test_spearman_sum = 0.0
            for idx, _pred in enumerate(test_prediction):
                rho, _ = spearmanr(histone_mark_day0[10000+idx], _pred)
                test_spearman_sum += rho
            print "Test spearman: {}".format(test_spearman_sum * 1.0 / 1000)

In [27]:
# CallBack: store bigwig file for the best model
class SaveBigwig(Callback):
    def __init__(self, X_train, y_train, X_val, y_val, X_test, y_test):
        self.best_val_loss = float('Inf')
        self.best_epoch = -1
        self.X_train, self.y_train = X_train, y_train
        self.X_val, self.y_val = X_val, y_val
        self.X_test, self.y_test = X_test, y_test
        self.epochs = 0
    
    def on_epoch_end(self, batch, logs={}):
        self.epochs += 1
        cur_val_loss = logs['val_loss']
        if cur_val_loss < self.best_val_loss:
            self.best_val_loss = cur_val_loss
            self.best_epoch = self.epochs
            prediction = model.predict(X_train).reshape(7500, window_size)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[:7500], os.path.join(srv_dir, 'train'))
            prediction = model.predict(X_val).reshape(2500, window_size)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[7500:10000], os.path.join(srv_dir, 'val'))
            prediction = model.predict(X_test).reshape(1000, window_size)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[10000:11000], os.path.join(srv_dir, 'test'))
            f = open(os.path.join(srv_dir, 'meta.txt'), 'wb')
            f.write(str(self.epochs) + "  " + str(logs))
            f.close()

In [28]:
# CallBack: Save model checkpoint based on validation loss
checkpointer = ModelCheckpoint(os.path.join(model_dir, "best_model.h5"),
                               monitor='val_loss',
                               verbose=1,
                               save_best_only=True,
                               mode='min')

In [29]:
print "Fitting the model..."
X_train, y_train = atac_seq_day0[:7500], histone_mark_day0[:7500]
X_val, y_val = atac_seq_day0[7500:10000], histone_mark_day0[7500:10000]
X_test, y_test = atac_seq_day0[10000:11000], histone_mark_day0[10000:11000]

model.fit(atac_seq_day0[:10000],
          histone_mark_day0[:10000],
          batch_size=batch_size,
          epochs=num_epochs,
          validation_split=0.25,   
          #shuffle=True,
          callbacks=[tensorboard,
                     reduce_lr,
                     EvaluateModel(X_test, y_test),
                     checkpointer,
                     SaveBigwig(X_train, y_train, X_val, y_val, X_test, y_test)])

Fitting the model...
Train on 7500 samples, validate on 2500 samples
Epoch 1/300
7424/7500 [============================>.] - ETA: 0s - loss: 236.4514 - pearson: 0.1594 - mean_squared_error: 236.4514 - mean_absolute_error: 5.4388
Epoch 00001: val_loss improved from inf to 97.39142, saving model to models/mse_lowlr_window4000/best_model.h5
Writing 1D track of shape: (7500, 5001)
Writing to file: /users/jesikmin/mse_lowlr_window4000/train.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (2500, 5001)
Writing to file: /users/jesikmin/mse_lowlr_window4000/val.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (1000, 5001)
Writing to file: /users/jesikmin/mse_lowlr_window4000/test.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
7500/7500 [==============================] - 454s 61ms/step - loss: 236.7354 - pearson: 0.1597 - mean_squared_error: 236.7354 - mean_absolute_error: 5.4358 - val_loss: 97.3914 - val_pearson: 0.1699 - 

Epoch 14/300
7424/7500 [============================>.] - ETA: 0s - loss: 217.8327 - pearson: 0.2243 - mean_squared_error: 217.8327 - mean_absolute_error: 5.2768
Epoch 00014: val_loss did not improve
7500/7500 [==============================] - 81s 11ms/step - loss: 218.4121 - pearson: 0.2240 - mean_squared_error: 218.4121 - mean_absolute_error: 5.2777 - val_loss: 95.9401 - val_pearson: 0.1782 - val_mean_squared_error: 95.9401 - val_mean_absolute_error: 3.0535
Epoch 15/300
7424/7500 [============================>.] - ETA: 0s - loss: 217.6551 - pearson: 0.2222 - mean_squared_error: 217.6551 - mean_absolute_error: 5.3399
Epoch 00015: val_loss did not improve
7500/7500 [==============================] - 80s 11ms/step - loss: 219.2571 - pearson: 0.2213 - mean_squared_error: 219.2571 - mean_absolute_error: 5.3506 - val_loss: 96.1143 - val_pearson: 0.1770 - val_mean_squared_error: 96.1143 - val_mean_absolute_error: 2.8358
Epoch 16/300
7424/7500 [============================>.] - ETA: 0s - lo

7424/7500 [============================>.] - ETA: 0s - loss: 215.6837 - pearson: 0.2348 - mean_squared_error: 215.6837 - mean_absolute_error: 5.3358
Epoch 00029: val_loss did not improve
7500/7500 [==============================] - 81s 11ms/step - loss: 216.3115 - pearson: 0.2368 - mean_squared_error: 216.3115 - mean_absolute_error: 5.3351 - val_loss: 95.4852 - val_pearson: 0.1820 - val_mean_squared_error: 95.4852 - val_mean_absolute_error: 2.8931
Epoch 30/300
1000/1000 [==============================] - 4s 4ms/steposs: 217.7902 - pearson: 0.233
[('loss', 118.66881744384766), ('pearson', 0.27806109058856965), ('mean_squared_error', 118.66881889343261), ('mean_absolute_error', 4.1299417400360108)]
Test spearman: 0.317363029254

Epoch 00030: val_loss did not improve
7500/7500 [==============================] - 91s 12ms/step - loss: 216.3768 - pearson: 0.2336 - mean_squared_error: 216.3768 - mean_absolute_error: 5.2816 - val_loss: 95.4955 - val_pearson: 0.1852 - val_mean_squared_error: 95

Epoch 43/300
7424/7500 [============================>.] - ETA: 0s - loss: 216.6489 - pearson: 0.2320 - mean_squared_error: 216.6489 - mean_absolute_error: 5.5354
Epoch 00043: val_loss did not improve
7500/7500 [==============================] - 81s 11ms/step - loss: 215.6329 - pearson: 0.2320 - mean_squared_error: 215.6329 - mean_absolute_error: 5.5256 - val_loss: 95.7762 - val_pearson: 0.1780 - val_mean_squared_error: 95.7762 - val_mean_absolute_error: 2.9490
Epoch 44/300
7424/7500 [============================>.] - ETA: 0s - loss: 215.4599 - pearson: 0.2342 - mean_squared_error: 215.4599 - mean_absolute_error: 5.2507
Epoch 00044: val_loss did not improve
7500/7500 [==============================] - 82s 11ms/step - loss: 215.3771 - pearson: 0.2329 - mean_squared_error: 215.3771 - mean_absolute_error: 5.2497 - val_loss: 96.0120 - val_pearson: 0.1783 - val_mean_squared_error: 96.0120 - val_mean_absolute_error: 2.7221
Epoch 45/300
7424/7500 [============================>.] - ETA: 0s - lo

7424/7500 [============================>.] - ETA: 0s - loss: 215.2681 - pearson: 0.2375 - mean_squared_error: 215.2681 - mean_absolute_error: 5.3294
Epoch 00058: val_loss did not improve
7500/7500 [==============================] - 81s 11ms/step - loss: 214.0368 - pearson: 0.2369 - mean_squared_error: 214.0368 - mean_absolute_error: 5.3210 - val_loss: 94.7809 - val_pearson: 0.1840 - val_mean_squared_error: 94.7809 - val_mean_absolute_error: 3.0558
Epoch 59/300
7424/7500 [============================>.] - ETA: 0s - loss: 215.0455 - pearson: 0.2407 - mean_squared_error: 215.0455 - mean_absolute_error: 5.3350
Epoch 00059: val_loss did not improve
7500/7500 [==============================] - 81s 11ms/step - loss: 214.0301 - pearson: 0.2408 - mean_squared_error: 214.0301 - mean_absolute_error: 5.3341 - val_loss: 95.0984 - val_pearson: 0.1841 - val_mean_squared_error: 95.0984 - val_mean_absolute_error: 3.2688
Epoch 60/300
1000/1000 [==============================] - 4s 4ms/steposs: 214.2155 

...

7424/7500 [============================>.] - ETA: 0s - loss: 209.3089 - pearson: 0.2529 - mean_squared_error: 209.3089 - mean_absolute_error: 5.3286
Epoch 00226: val_loss did not improve
7500/7500 [==============================] - 81s 11ms/step - loss: 208.9127 - pearson: 0.2538 - mean_squared_error: 208.9127 - mean_absolute_error: 5.3220 - val_loss: 95.9538 - val_pearson: 0.1749 - val_mean_squared_error: 95.9538 - val_mean_absolute_error: 2.8838
Epoch 227/300
7424/7500 [============================>.] - ETA: 0s - loss: 209.4338 - pearson: 0.2559 - mean_squared_error: 209.4338 - mean_absolute_error: 5.2320
Epoch 00227: val_loss did not improve
7500/7500 [==============================] - 80s 11ms/step - loss: 208.9207 - pearson: 0.2564 - mean_squared_error: 208.9207 - mean_absolute_error: 5.2247 - val_loss: 95.5840 - val_pearson: 0.1766 - val_mean_squared_error: 95.5840 - val_mean_absolute_error: 2.9738
Epoch 228/300
7424/7500 [============================>.] - ETA: 0s - loss: 209.527

7424/7500 [============================>.] - ETA: 0s - loss: 209.5422 - pearson: 0.2523 - mean_squared_error: 209.5422 - mean_absolute_error: 5.3040
Epoch 00243: val_loss did not improve
7500/7500 [==============================] - 81s 11ms/step - loss: 208.8070 - pearson: 0.2543 - mean_squared_error: 208.8070 - mean_absolute_error: 5.3039 - val_loss: 96.0302 - val_pearson: 0.1736 - val_mean_squared_error: 96.0302 - val_mean_absolute_error: 2.9663
Epoch 244/300
7424/7500 [============================>.] - ETA: 0s - loss: 210.0813 - pearson: 0.2574 - mean_squared_error: 210.0813 - mean_absolute_error: 5.1647
Epoch 00244: val_loss did not improve
7500/7500 [==============================] - 81s 11ms/step - loss: 208.9949 - pearson: 0.2579 - mean_squared_error: 208.9949 - mean_absolute_error: 5.1568 - val_loss: 95.5285 - val_pearson: 0.1772 - val_mean_squared_error: 95.5285 - val_mean_absolute_error: 3.0237
Epoch 245/300
7424/7500 [============================>.] - ETA: 0s - loss: 209.010

1000/1000 [==============================] - 4s 4ms/steposs: 208.1749 - pearson: 0.252
[('loss', 117.88912308502198), ('pearson', 0.27179164415597917), ('mean_squared_error', 117.88912100982667), ('mean_absolute_error', 3.6230553188323973)]
Test spearman: 0.305030470637

Epoch 00260: val_loss did not improve
7500/7500 [==============================] - 89s 12ms/step - loss: 208.9007 - pearson: 0.2534 - mean_squared_error: 208.9007 - mean_absolute_error: 5.2530 - val_loss: 95.6661 - val_pearson: 0.1762 - val_mean_squared_error: 95.6661 - val_mean_absolute_error: 2.9622
Epoch 261/300
7424/7500 [============================>.] - ETA: 0s - loss: 209.2786 - pearson: 0.2572 - mean_squared_error: 209.2786 - mean_absolute_error: 5.2424
Epoch 00261: val_loss did not improve
7500/7500 [==============================] - 80s 11ms/step - loss: 208.5473 - pearson: 0.2560 - mean_squared_error: 208.5473 - mean_absolute_error: 5.2314 - val_loss: 95.8167 - val_pearson: 0.1745 - val_mean_squared_error: 9

7424/7500 [============================>.] - ETA: 0s - loss: 208.4103 - pearson: 0.2591 - mean_squared_error: 208.4103 - mean_absolute_error: 5.2975
Epoch 00277: val_loss did not improve
7500/7500 [==============================] - 80s 11ms/step - loss: 208.3519 - pearson: 0.2587 - mean_squared_error: 208.3519 - mean_absolute_error: 5.2959 - val_loss: 95.4232 - val_pearson: 0.1776 - val_mean_squared_error: 95.4232 - val_mean_absolute_error: 3.0159
Epoch 278/300
7424/7500 [============================>.] - ETA: 0s - loss: 208.2733 - pearson: 0.2518 - mean_squared_error: 208.2733 - mean_absolute_error: 5.2519
Epoch 00278: val_loss did not improve
7500/7500 [==============================] - 81s 11ms/step - loss: 208.5444 - pearson: 0.2510 - mean_squared_error: 208.5444 - mean_absolute_error: 5.2596 - val_loss: 95.3385 - val_pearson: 0.1777 - val_mean_squared_error: 95.3385 - val_mean_absolute_error: 3.0149
Epoch 279/300
7424/7500 [============================>.] - ETA: 0s - loss: 208.571

7424/7500 [============================>.] - ETA: 0s - loss: 208.4294 - pearson: 0.2576 - mean_squared_error: 208.4294 - mean_absolute_error: 5.2582
Epoch 00294: val_loss did not improve
7500/7500 [==============================] - 80s 11ms/step - loss: 208.0964 - pearson: 0.2579 - mean_squared_error: 208.0964 - mean_absolute_error: 5.2559 - val_loss: 95.4313 - val_pearson: 0.1770 - val_mean_squared_error: 95.4313 - val_mean_absolute_error: 3.0369
Epoch 295/300
7424/7500 [============================>.] - ETA: 0s - loss: 207.2103 - pearson: 0.2550 - mean_squared_error: 207.2103 - mean_absolute_error: 5.2751
Epoch 00295: val_loss did not improve
7500/7500 [==============================] - 80s 11ms/step - loss: 208.4837 - pearson: 0.2546 - mean_squared_error: 208.4837 - mean_absolute_error: 5.2874 - val_loss: 95.9016 - val_pearson: 0.1739 - val_mean_squared_error: 95.9016 - val_mean_absolute_error: 2.9607
Epoch 296/300
7424/7500 [============================>.] - ETA: 0s - loss: 208.408